In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [2]:
dataset = 'emma' # onion or emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_emma.tsv', 'id_gems.tsv'],
    'audio_embeddings': ['id_maest.tsv'],# 'id_jukebox.tsv', 'id_musicnn.tsv'],
    'textual_embeddings': ['id_bert.tsv'],# 'id_lyrics_word2vec.tsv'],
    'visual_embeddings': ['id_resnet.tsv'],# 'id_vgg19.tsv', 'id_incp.tsv'],
}

output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [3]:
df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes


print(df_interactions.shape)
df_interactions.head()

(10946, 8)


,user_id,track_id,timestamp,year,month,rating,user_id_int,track_id_int
0,13288,yfm7RER1PWTqgjIp,2014-03-16 18:31:51,2014,2014-03,5,1330,205
1,69279,qnmsOhb7GYwNNC4S,2014-03-25 22:43:31,2014,2014-03,5,4885,170
2,13764,yfm7RER1PWTqgjIp,2014-02-13 06:23:23,2014,2014-02,5,1365,205
3,46933,yfm7RER1PWTqgjIp,2014-04-05 00:16:39,2014,2014-04,5,3717,205
4,70060,yfm7RER1PWTqgjIp,2014-04-03 11:31:02,2014,2014-04,5,4918,205


In [4]:
# track ids from feature file
df = pd.read_csv(input_data_dir + features['emotion_embeddings'][1], sep='\t')
feat_track_ids = df['id'].values
feat_track_ids

array(['04q3VppIQEET5rzy', '07xF9Q0K1t3ist7K', '0JbYcELIqpMPvAHk', ...,
       'zzyyPUs7hC9Nz2e1', 'zzz0n04uuTUA7fNh', 'zzzj3LYaZtYtbzSr'],
      dtype=object)

In [5]:
# remove interactions with items not in the feature files
df_interactions = df_interactions[df_interactions['track_id'].isin(feat_track_ids)]
df_interactions.shape

(10946, 8)

In [6]:
#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

In [7]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
yfm7RER1PWTqgjIp,205
qnmsOhb7GYwNNC4S,170
cwXiTYqrj7AOGn7x,129
TwBJLxtsSwb6lS6c,101
zbI3TrMSY2chqzRz,207


In [8]:
del df_interactions

In [9]:
for feature_type in features:

    for file in features[feature_type]:
        if dataset == 'onion' and 'emma' in file:
            continue

        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values[:-1])

Processing id_emma.tsv


210it [00:00, 5674.99it/s]

Processing id_gems.tsv



210it [00:00, 8875.58it/s]

Processing id_maest.tsv



210it [00:00, 7706.74it/s]


Processing id_bert.tsv


210it [00:00, 7257.90it/s]


Processing id_resnet.tsv


210it [00:00, 5034.80it/s]
